In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from ast import literal_eval
import warnings; warnings.simplefilter('ignore')


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/tmdb-movies-dataset-2023-930k-movies")

print("Path to dataset files:", path)
df = pd. read_csv(f"{path}/TMDB_movie_dataset_v11.csv")
df.head()
#keep only 60% of this data and remove all


Path to dataset files: /root/.cache/kagglehub/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/versions/518


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [ ]:
df.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
206958,846218,Upon Her Lips: Heartbeats,5.5,2,Released,2021-06-06,0,89,False,NaN,...,Upon Her Lips: Heartbeats,Witness and explore a whole range of emotions ...,1.031,/5RPqiyA6Brzkp5SCBkbtED5dR1a.jpg,NaN,[],NQV Media,[],[],[]
464122,1093609,The Whiteboxxx 43,0.0,0,Released,2019-07-01,0,0,True,NaN,...,The Whiteboxxx 43,,0.000,NaN,NaN,[],#LetsDoeit,[],[],[]
53176,163036,Snow Business,6.3,18,Released,1953-01-17,0,7,False,/pT5hUXouLciJkZFm2FNcVXSiluX.jpg,...,Snow Business,Sylvester Cat and Tweety Bird are snowbound in...,2.193,/jfUVEZcb9yqbAHr3V2MrBmx4bLN.jpg,NaN,"[animation, comedy]",Warner Bros. Cartoons,[unitedstatesofamerica],[english],"[cartoon, canary, food, cartooncat, cartoonmou..."
209800,254951,Moster fra Mols,5.0,2,Released,1943-02-24,0,0,False,NaN,...,Moster fra Mols,,0.756,/8Q17iEtnJYCnlPwZoqOe22j1Eej.jpg,NaN,"[comedy, family]",ASA,[denmark],[danish],[]
221872,266359,Zoo,6.0,2,Released,1988-11-17,0,87,False,/l7cGMlnP10E2yyk6367uaCR2E7h.jpg,...,Zoo,"Martina lives in the Rome's zoo, because her f...",0.840,/4jaXblcXFkRg99Vicq7t1rRgV4C.jpg,NaN,"[comedy, drama]","Ital-Noleggio Cinematografico, Reteitalia, Veg...",[italy],[italian],"[zoo, gipsy, womandirector, slavic]"


In [ ]:
print(f"DataFrame Shape: {df.shape}")
print("Columns:", df.columns.tolist())
df.isnull().sum()

DataFrame Shape: (717494, 24)
Columns: ['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date', 'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'tagline', 'genres', 'production_companies', 'production_countries', 'spoken_languages', 'keywords']


,0
id,0
title,6
vote_average,0
vote_count,0
status,0
release_date,127627
revenue,0
runtime,0
adult,0
backdrop_path,529433


In [ ]:
features = ['keywords', 'genres', 'overview', 'title', 'spoken_languages', 'production_countries']
df[features] = df[features].fillna('')
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)

143


In [ ]:
#find movie title with given production houses
# df[df['production_companies'].str.contains('#LetsDoeit')]
#pritn  title  with null tagline
df[df['production_countries'].isnull()]['title']

,title
813,Salt
2833,Gabriel's Inferno: Part II
5530,Return
6011,Return
6536,Status Update
...,...
1195818,Adam Hess - Seahorse
1195819,Ali Brice - Bin Wondering
1195820,Andrew Maxwell - Showtime
1195821,Erotic StreetLife 35 - Buddy & The Wet Slips
